# notebook for further prototyping:

why I need to include intent='inout' for state, and have to add init process:

In [1]:
import numpy as np
import xsimlab as xs

In [2]:
# first build 1.
@xs.process
class OneState:
    
    state = xs.variable(intent='inout')
    
    fluxes = xs.group('flux')
    
    def initialize(self):
        self.state=1
    
    @xs.runtime(args="step_delta")
    def run_step(self, dt):
        self.delta = sum((v for v in self.fluxes)) * dt  # multiply by time step

    def finalize_step(self):
        self.state += self.delta

@xs.process
class OtherState(OneState):
    state = xs.variable(intent='out')
    
    
@xs.process
class OneFlux:
    
    flux1 = xs.variable(intent='out', groups='flux')
    
    state1 = xs.foreign(OneState,'state')
    
    def run_step(self):
        self.flux1 = 0.1 * self.state1

In [3]:
# This is necessary when setting state intent 'inout', and reusing it in the Flux calculation as xs.foreign

@xs.process
class Init:
    
    state = xs.foreign(OneState,'state', intent='out')
    
    def initialize(self):
        self.state = 0.1

## this works:

In [4]:
MinimalStateFluxModel = xs.Model({'State':OneState, 'Flux':OneFlux, 'Setup':Init})

### this gives a cycle dependency error:

In [5]:
out_MinimalStateFluxModel = xs.Model({'State':OtherState, 'Flux':OneFlux})

RuntimeError: Cycle detected in process graph: State->Flux->State

### Model runs fine & as expected with intent=`inout`

In [6]:
in_ds = xs.create_setup(
    model=MinimalStateFluxModel,
    clocks={   
        'time': np.linspace(0,2,30),
    },
    master_clock='time',
    #input_vars={'State__state':0.2},
    output_vars={'State__state':'time'}
                 )

In [7]:
with MinimalStateFluxModel:
    out_ds = in_ds.xsimlab.run()

In [8]:
out_ds

<xarray.Dataset>
Dimensions:       (time: 30)
Coordinates:
  * time          (time) float64 0.0 0.06897 0.1379 0.2069 ... 1.862 1.931 2.0
Data variables:
    State__state  (time) int64 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1

# further prototyping:

Questions:
1. how to best handle the process class constructor, for my purposes 
(i.e. to be able to flexibily initialize base components in different dimensions)

# below here, test process class factory:

i.e. Question 1

IDEA: I can use test-case dimensions!

PhysEnv collects label and state of each component, via xs.group!()

and distributes fluxes from there! it collects forcing fluxes, and passes them down (thanks to xs.group() labels.

as base component, something like this:
@xs.process
class Component:
    """Base class for a component of a ecosystem."""

    label = xs.variable(groups='c_labels', description='component label')
    
    c0 = xs.variable(
        # only slab model (scalar)
        dims=(), 
        # support slab + 2D models ?
        #dims=[(), ('lat', 'lon')],
        groups='c_c0',
        description='inital concentration'
    )

# also the class factory

In [9]:
## now how to combine this in a meaningful way:

- PhysEnv collects array of components and handles forcing fluxes and parameters
(perhaps these parameters can be handled flexibly via labels & dicts)

- Class Factory sets up Basic Components, so that this flexibility is possible

SyntaxError: invalid syntax (<ipython-input-9-d219152b4390>, line 3)

In [10]:
# first step:
# get a simple class factory to work

import attr

@xs.process
class BasePhyto:
    
    TestVar = xs.variable(default=1)
    
    def initialize(self):
        print(self.TestVar)
        
    @TestVar.validator
    def _check_TestVar(self, attribute, value):
        if not np.isscalar(value) and len(value) != 4:
            raise ValueError(
                "Border status should be defined for all borders "
                f"(left, right, top, bottom), found {value}"
            )

In [11]:
attrib_dict = attr.fields_dict(BasePhyto)#.items()

In [18]:
attrib_dict = attr.fields_dict(OneState)

In [19]:
attrib_dict

{'state': Attribute(name='state', default=NOTHING, validator=None, repr=True, eq=True, order=True, hash=None, init=True, metadata=mappingproxy({'var_type': <VarType.VARIABLE: 'variable'>, 'dims': ((),), 'intent': <VarIntent.INOUT: 'inout'>, 'groups': (), 'static': False, 'attrs': {}, 'description': '', 'encoding': {}}), type=None, converter=None, kw_only=True),
 'fluxes': Attribute(name='fluxes', default=(), validator=None, repr=True, eq=True, order=True, hash=None, init=True, metadata=mappingproxy({'var_type': <VarType.GROUP: 'group'>, 'group': 'flux', 'intent': <VarIntent.IN: 'in'>, 'description': "Iterable of all variables that belong to group 'flux'"}), type=None, converter=None, kw_only=True)}

In [20]:
new_cls = attr.make_class(
         'new_cls_name',
        attrs=attrib_dict,
        bases=(BasePhyto,),
        init=False,
        repr=False
    )

AttributeError: 'Attribute' object has no attribute '_validator'

In [15]:
def make_phyto_cls(new_cls_name): #, arg1, arg2):
    attrib_dict = attr.fields_dict(BasePhyto).items()

    #attrib_dict.update(...)   # some logic to re-define some variables

    new_cls = attr.make_class(
        new_cls_name, 
        attrib_dict,
        bases=(BasePhyto,),
        init=False,
        repr=False
    )

    return xs.process(new_cls)

In [16]:
arg1 = [1]
arg2 = [2]

In [17]:
my_model = xs.Model({
    'p1': make_phyto_cls('Phyto1'), #, arg1, arg2),
    'p2': make_phyto_cls('Phyto2') #, arg1, arg2)
})

TypeError: attrs argument must be a dict or a list.